In [1]:
from pathlib import Path
from typing import List

from IPython.display import display
from ipywidgets import widgets

import logging

logger = logging.getLogger()

out = widgets.Output()

from dashboard_funcs import (
    create_start_standalone_command,
    write_shell_file,
    s3_download_file,
    write_fews_desktop_shortcut,
    s3_download_directory
)

BUCKET_NAME = "ciroh-rti-hefs-data"

FEWS_INSTALL_DIR = Path("/opt", "fews")



In [2]:
# List the unique RFCs in the s3 bucket.
unique_rfcs = [
    "ABRFC",
    "APRFC",
    "CBRFC",
    "CNRFC",
    "LMRFC",
    "MARFC",
    "MBRFC",
    "NCRFC",
    "NERFC",
    "NWRFC",
    "OHRFC",
    "SERFC",
    "WGRFC"
]

In [3]:
rfc_dropdown = widgets.Dropdown(
    options=unique_rfcs,
    description='RFC:',
    disabled=False,
    value="MARFC",
)

In [4]:
style = {'description_width': 'initial'}
download_dir_textbox = widgets.Text(
    value='/opt/data',
    placeholder='Type something',
    description='Download Directory:',
    disabled=False,
    style=style
)

In [5]:
@out.capture(clear_output=True, wait=True)
def install_fews_standalone(b) -> None:
    """Download standalone configuration from S3 to the working directory."""
    # print(download_dir_textbox.value)
    fews_download_dir = Path(download_dir_textbox.value)
    if not fews_download_dir.exists():
        print(f"The directory: {fews_download_dir}, does not exist. Please create it first!")
        raise ValueError(f"The directory: {fews_download_dir}, does not exist. Please create it first!")

    # 1. Download sa from S3
    print(f"Downloading RFC configuration to {fews_download_dir.as_posix()}...This will take a few minutes...")
    s3_download_directory(
        prefix=f"{rfc_dropdown.value}/Config",
        local=fews_download_dir
    )
    # 2. Create the bash command to run the standalone configuration
    sa_dir_path = Path(fews_download_dir, rfc_dropdown.value)
    bash_command_str = create_start_standalone_command(
        fews_root_dir=FEWS_INSTALL_DIR.as_posix(),
        configuration_dir=sa_dir_path.as_posix()
    )
    # 3. Write the command to start FEWS to a shell script
    shell_script_filepath = Path(sa_dir_path, "start_fews_standalone.sh")
    write_shell_file(shell_script_filepath, bash_command_str)
    # 4. Copy in patch file for the downloaded standalone config.
    print("Downloading patch file...")
    s3_download_file(
        remote_filepath="fews-install/fews-NA-202102-125264-patch.jar",
        local_filepath=Path(sa_dir_path, "fews-NA-202102-125264-patch.jar")
    )
    # 5. Create FEWS desktop shortcut that calls the shell script
    desktop_shortcut_filepath = Path(Path.home(), "Desktop", f"{sa_dir_path.name}.desktop")
    print(f"Creating FEWS desktop shortcut...{desktop_shortcut_filepath}")
    write_fews_desktop_shortcut(desktop_shortcut_filepath, shell_script_filepath)
    print("Installation complete.")
    return

In [6]:
@out.capture(clear_output=True, wait=True)
def download_historical_data(b) -> None:
    """Download historical data for selected RFC."""
    # print(download_dir_textbox.value)
    fews_download_dir = Path(download_dir_textbox.value)
    if not fews_download_dir.exists():
        print(f"The directory: {fews_download_dir}, does not exist. Please create it first!")
        raise ValueError(f"The directory: {fews_download_dir}, does not exist. Please create it first!")

    print(f"Downloading historical data to {fews_download_dir.as_posix()}...")
    s3_download_directory(
        prefix=f"{rfc_dropdown.value}/historicalData",
        local=fews_download_dir
    )
    local_path = Path(fews_download_dir, rfc_dropdown.value, "historicalData")
    local_path.rename(Path(fews_download_dir, rfc_dropdown.value, "cardfiles"))
    pass

In [7]:
data_button = widgets.Button(description='Download Data')
data_button.on_click(download_historical_data)

In [8]:
config_button = widgets.Button(description='Download Config')
config_button.on_click(install_fews_standalone)

![CIROH Logo](CIROHLogo_200x200.png)

## Welcome to the HEFS-FEWS TEEHR-Hub Dashboard!

### This experimental dashboards allows you to download and install pre-existing FEWS stand-alone configurations from the ciroh-rti-hefs-data S3 bucket, and a desktop shortcut to open FEWS.

Configure FEWS for your region of interest:

In [9]:
display(rfc_dropdown)

Dropdown(description='RFC:', index=5, options=('ABRFC', 'APRFC', 'CBRFC', 'CNRFC', 'LMRFC', 'MARFC', 'MBRFC', …

In [10]:
display(download_dir_textbox)

Text(value='/opt/data', description='Download Directory:', placeholder='Type something', style=TextStyle(descr…

In [11]:
display(config_button)

Button(description='Download Config', style=ButtonStyle())

In [12]:
display(data_button)

Button(description='Download Data', style=ButtonStyle())

In [13]:
display(out)

Output()